In [ ]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
import urllib.parse
import pandas as pd
from pycaret.clustering import *

badwords = ['sleep', 'drop', 'select', 'waitfor', 'delay', 'system', 'union', 'order by', 'group by']

def ExtractFeatures(path):
    path = urllib.parse.unquote(path)
    badwords_count = 0
    single_q = path.count("'")
    double_q = path.count("\"")
    dashes = path.count("--")
    braces = path.count("(")
    spaces = path.count(" ")
    for word in badwords:
        badwords_count += path.count(word)
    lst = [single_q, double_q, dashes, braces, spaces, badwords_count]
    print(lst)
    return pd.DataFrame([lst], columns=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords'])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
http = pd.read_excel('/content/drive/MyDrive/ColabNotebooks/mini_project_7th/allData.xlsx')
clu1 = setup(data = http, normalize = True, numeric_features = ['single_q', 'double_q', 'braces', 'spaces', 'badwords'], ignore_features = ['method', 'path', 'body', 'class' ])
kmeans = create_model('kmeans', num_clusters = 2)

,Description,Value
0,Session id,7743
1,Original data shape,"(526, 9)"
2,Transformed data shape,"(526, 5)"
3,Ignore features,4
4,Numeric features,5
5,Rows with missing values,45.6%
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6572,135.1242,1.4404,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'tsne')

In [ ]:
kmeans_result.to_csv('/content/kmeans_result.csv')
from google.colab import files
files.download('/content/kmeans_result.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_http = pd.read_csv('/content/kmeans_result.csv')
test_http.head()


In [ ]:
import http.server
import urllib.request
import urllib.error

class SimpleHTTPProxy(http.server.SimpleHTTPRequestHandler):
    proxy_routes = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        parts = self.proxy_routes
        print(parts)
        live_data = ExtractFeatures(parts.get('proxy_route', ''))
        result = predict_model(kmeans, data=live_data)
        print(result['Cluster'][0])
        if result['Cluster'][0] == "Cluster 1":
            print('Intrusion Detected')
        if len(parts) >= 2:
            self.proxy_request('http://' + parts['proxy_route'] + '/')
        else:
            super().do_GET()

    def proxy_request(self, url):
        try:
            response = urllib.request.urlopen(url)
        except urllib.error.HTTPError as e:
            print('err')
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)

SimpleHTTPProxy.set_routes({'proxy_route': 'https://demo.testfire.net/index.jsp?content=personal_savings.htm'})
with http.server.HTTPServer(('127.0.0.1', 8081), SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print(f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nKeyboard interrupt received, exiting.")


Listening on http://127.0.0.1:8081
